In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import preprocess as pre
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

In [2]:
# --- Data Preprocessing --- 
start = time.time()
pre.ais_trainCleanup("../CSV/big_files/ais_train.csv", "ais_train_processed.csv")
end = time.time()

print(end-start)

c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["time"] = boat["time"].astype("datetime64[ns]")
c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["drift"] = boat["cog"] - boat["heading"]
c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

4.894385099411011


In [11]:
data = pd.read_csv("../CSV/big_files/ais_train_processed.csv", sep="|")
data.drop(columns=["portId"], inplace=True)

X = data.copy().drop(columns=["latitude", "longitude"])
y = data.copy()[["latitude", "longitude"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = pd.get_dummies(X_train, columns=["vesselId"], drop_first=True)
X_test = pd.get_dummies(X_test, columns=["vesselId"], drop_first=True)

#More data BS
for col in X_train:
    if col not in X_test:
        X_test[col] = False

X_train = X_train.reindex(sorted(X_train.columns), axis=1)
X_test = X_test.reindex(sorted(X_test.columns), axis=1)




,latitude,longitude
0,-34.74370,-57.85130
1,8.89440,-79.47939
2,39.19065,-76.47567
3,-34.41189,151.02067
4,35.88379,-5.91636


In [10]:
def evaluate(model: str, y_pred):

    n = len(y_test)
    p = X_test.shape[1]

    mse_latitude = mean_squared_error(y_test, y_pred)
    mse_longitude = mean_squared_error(y_test, y_pred)
    r2_latitude = r2_score(y_test, y_pred)
    r2_latitude_adj = 1 - (1 - r2_latitude) * ((n - 1) / (n - p - 1))
    r2_longitude = r2_score(y_test, y_pred)
    r2_longitude_adj = 1 - (1 - r2_longitude) * ((n - 1) / (n - p - 1))


    print(f"---- {model} Metrics ----")
    print(f"Mean Squared Error (Latitude): {mse_latitude:.4f}")
    print(f"R-squared (Latitude): {r2_latitude:.4f}, Adjusted R-squared (Latitude): {r2_latitude_adj: .4f}")
    print(f"Mean Squared Error (Longitude): {mse_longitude:.4f}")
    print(f"R-squared (Longitude): {r2_longitude:.4f}, Adjusted R-squared (Latitude): {r2_longitude_adj: .4f}")


In [ ]:
# --- LINEAR REGRESSION ---
linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg_pred = linreg.predict(X_test)

#Metrics
evaluate("Linear Regression", linreg_pred)

In [ ]:
# --- XGBOOST ---
xgbmodel = xgb.XGBRegressor()
xgbmodel.fit(X_train, y_train)
xgb_pred = xgbmodel.predict(X_test)

evaluate("XGBoost", xgb_pred)

In [12]:
# --- Random Forest ---
from sklearn.ensemble import RandomForestRegressor

start = time.time()
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
rfr_pred = rfr.predict(X_test)
print(time.time()-start)

evaluate("Random Forest", rfr_pred)



,ID,vesselId,time,scaling_factor
0,0,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:03:16,0.3
1,1,61e9f473b937134a3c4c02df,2024-05-08 00:06:17,0.3
2,2,61e9f469b937134a3c4c029b,2024-05-08 00:10:02,0.3
3,3,61e9f45bb937134a3c4c0221,2024-05-08 00:10:34,0.3
4,4,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,0.3
